---
## Setup and Variables

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import gc
import copy
import random

import torch
import torch.nn as nn
import pandas as pd
import numpy as np

import seaborn as sns

import evaluate

from transformers import (
    T5Tokenizer,
    DataCollatorForTokenClassification,
    TrainingArguments,
    Trainer,
    TrainerCallback
)

from src.model_new import (
    T5EncoderModelForTokenClassification,
)
import src.config
import src.data
import src.model_new


import peft
from peft import (
    LoraConfig,
)

import src.utils

import sklearn.metrics

In [3]:
print("Base Model:\t", src.config.base_model_name)
print("MPS:\t\t", torch.backends.mps.is_available())
ROOT = src.utils.get_project_root_path()
print("Path:\t\t", ROOT)
device = torch.device('cuda:0' if torch.cuda.is_available() else ('mps' if torch.backends.mps.is_available() else 'cpu'))
print(f"Using device:\t {device}")

torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

Base Model:	 Rostlab/prot_t5_xl_uniref50
MPS:		 False
Path:		 /home/ec2-user/developer/prottrans-t5-signalpeptide-prediction
Using device:	 cuda:0


---
## Create Tokenizer and Load Model

In [4]:
t5_tokenizer = T5Tokenizer.from_pretrained(
        pretrained_model_name_or_path=src.config.base_model_name,
        do_lower_case=False,
        use_fast=True,
        legacy=False
    )

In [5]:
t5_base_model = T5EncoderModelForTokenClassification.from_pretrained(
    pretrained_model_name_or_path=src.config.base_model_name,
    device_map='auto',
    load_in_8bit=False,
    custom_num_labels=len(src.config.label_decoding),
    custom_dropout_rate=0.1,
    )

Some weights of T5EncoderModelForTokenClassification were not initialized from the model checkpoint at Rostlab/prot_t5_xl_uniref50 and are newly initialized: ['custom_classifier.weight', 'custom_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
t5_base_model.custom_classifier.weight.data = nn.Linear(
    in_features=t5_base_model.config.hidden_size,
    out_features=t5_base_model.custom_num_labels).weight.data

In [7]:
t5_base_model.custom_classifier.weight.data

tensor([[ 0.0239,  0.0259, -0.0073,  ..., -0.0119, -0.0269, -0.0198],
        [ 0.0174, -0.0047,  0.0133,  ...,  0.0008,  0.0057,  0.0295],
        [ 0.0145,  0.0067, -0.0063,  ...,  0.0145, -0.0186,  0.0159],
        [-0.0079,  0.0028, -0.0084,  ...,  0.0109, -0.0184,  0.0002],
        [-0.0221,  0.0252,  0.0264,  ...,  0.0257, -0.0213,  0.0044],
        [-0.0065, -0.0306, -0.0304,  ..., -0.0284, -0.0211, -0.0068]])

In [ ]:
# t5_base_model_copy = copy.deepcopy(t5_base_model)

---
## Apply LoRA

In [8]:
lora_config = LoraConfig(
    inference_mode=False,
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=['q', 'k', 'v', 'o'],
    bias="none",
    modules_to_save=['custom_classifier'],
)

t5_lora_model = peft.get_peft_model(t5_base_model, lora_config)
t5_lora_model.print_trainable_parameters()

trainable params: 3,944,460 || all params: 1,212,086,284 || trainable%: 0.32542732741623864


In [9]:
t5_lora_model.base_model.custom_classifier.modules_to_save.default.weight.data

tensor([[ 0.0239,  0.0259, -0.0073,  ..., -0.0119, -0.0269, -0.0198],
        [ 0.0174, -0.0047,  0.0133,  ...,  0.0008,  0.0057,  0.0295],
        [ 0.0145,  0.0067, -0.0063,  ...,  0.0145, -0.0186,  0.0159],
        [-0.0079,  0.0028, -0.0084,  ...,  0.0109, -0.0184,  0.0002],
        [-0.0221,  0.0252,  0.0264,  ...,  0.0257, -0.0213,  0.0044],
        [-0.0065, -0.0306, -0.0304,  ..., -0.0284, -0.0211, -0.0068]])

---
## Load Data, Split into Dataset, and Tokenize Sequences

In [7]:
FASTA_FILENAME = '5_SignalP_5.0_Training_set.fasta'
# FASTA_FILENAME = '5_SignalP_5.0_Training_set_testing.fasta'
annotations_name = 'Label' # Choose Type or Label

df_data = src.data.process(src.data.parse_file(ROOT + '/data/raw/' + FASTA_FILENAME))

dataset_signalp = src.model_new.create_datasets(
    splits=src.config.splits,
    tokenizer=t5_tokenizer,
    data=df_data,
    annotations_name=annotations_name,
    dataset_size=src.config.dataset_size,
    encoder=src.config.select_encodings[annotations_name],
    )

del df_data

In [8]:
# display(dataset_signalp)
# print(dataset_signalp['valid'][0]['input_ids'])
# print(dataset_signalp['valid'][0]['labels'])
# print(dataset_signalp['valid'][0]['attention_mask'])

---
## Training Loop
https://huggingface.co/docs/peft/task_guides/token-classification-lora

In [9]:
accuracy_metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")
# roc_auc_score_metric = evaluate.load("roc_auc", "multiclass")
matthews_correlation_metric = evaluate.load("matthews_correlation")

In [10]:
def batch_eval_elementwise(predictions: np.ndarray, references: np.ndarray):
    results = {}
    
    if np.isnan(predictions).any():
        print('has nan')
        predictions = np.nan_to_num(predictions)
    
    argmax_predictions = predictions.argmax(axis=-1)
    vals = list((np.array(p)[(r != -100)], np.array(r)[(r != -100)]) for p, r in zip(argmax_predictions.tolist(), references))

    lst1, lst2 = zip(*vals)
    confusion_matrix = sklearn.metrics.confusion_matrix(y_true=np.concatenate(lst1), y_pred=np.concatenate(lst2))
    
    results.update({'accuracy_metric': np.average([accuracy_metric.compute(predictions=x, references=y)['accuracy'] for x, y in vals])})
    results.update({'precision_metric': np.average([precision_metric.compute(predictions=x, references=y, average='micro')['precision'] for x, y in vals])})
    results.update({'recall_metric': np.average([recall_metric.compute(predictions=x, references=y, average='micro')['recall'] for x, y in vals])})
    results.update({'f1_metric': np.average([f1_metric.compute(predictions=x, references=y, average='micro')['f1'] for x, y in vals])})
    # results.update({'roc_auc': [roc_auc_score_metric.compute(prediction_scores=x, references=y, multi_class='ovr', average=None)['roc_auc'] for x, y in zip(softmax_predictions, references)]})
    results.update({'matthews_correlation': np.average([matthews_correlation_metric.compute(predictions=x, references=y, average='micro')['matthews_correlation'] for x, y in vals])})
    results.update({'confusion_matrix': confusion_matrix})

    return results

def compute_metrics(p):
    predictions, references = p
    results = batch_eval_elementwise(predictions=predictions, references=references)
    return results


In [11]:
# t5_lora_model.to(device)

In [12]:
# vals = 0,1,2
# with torch.no_grad():
#     preds = t5_lora_model(
#         input_ids=torch.tensor(dataset_signalp['valid'][vals]['input_ids']).to(device),
#         # attention_mask=torch.tensor(dataset_signalp['test'][0:1]['attention_mask']).to(device),
#         # labels=torch.tensor(dataset_signalp['test'][0:1]['labels']).to(device)
#         )

In [13]:
# _p = preds.logits.cpu().numpy()
# _t = np.array([np.pad(x, (0, 71 - len(x)), mode='constant', constant_values=-100) for x in dataset_signalp['valid'][vals]['labels']])
# metrics = compute_metrics(p=(_p, _t))
# print(metrics)

In [14]:
# ax = sns.heatmap(
#     metrics['confusion_matrix'],
#     annot=True,
#     xticklabels=[src.config.label_decoding[label] for label in range(len(src.config.label_decoding))],
#     yticklabels=[src.config.label_decoding[label] for label in range(len(src.config.label_decoding))],
#     )

# ax.set_title('Confusion Matrix')
# ax.set_xlabel('Actual')
# ax.set_ylabel('Predicted')

In [15]:
# pd.DataFrame(preds.logits.softmax(dim=-1).cpu()[0]).plot()

In [16]:
# roc_auc_score = evaluate.load("roc_auc", "multiclass")
# refs = [1, 0, 1, 2, 2, 0]
# pred_scores = [[0.3, 0.5, 0.2],
#                [0.7, 0.2, 0.1],
#                [0.005, 0.99, 0.005],
#                [0.2, 0.3, 0.5],
#                [0.1, 0.1, 0.8],
#                [0.1, 0.7, 0.2]]
# results = roc_auc_score.compute(references=refs,
#                                 prediction_scores=pred_scores,
#                                 multi_class='ovr')
# print(round(results['roc_auc'], 2))

In [17]:
# print(np.array(refs))
# print(np.array(pred_scores))

In [18]:
# p = preds.logits.cpu().numpy()
# t = np.array(dataset_signalp['valid'][0:1]['labels'])
# t = np.pad(t, ((0, 0), (0, 71 - t.shape[1])), mode='constant', constant_values=-100)
# compute_metrics(p=(p, t))

In [19]:
# x = 10
# truth = t[:, :x][0]
# truth = [1,2,3,4,5,6,5,5,5,5]
# preds = torch.tensor(p[:, :x][0]).softmax(axis=-1).cpu().numpy()
# print(truth)
# print(preds)

In [20]:

# results = roc_auc_score.compute(references=truth,
#                                 prediction_scores=preds,
#                                 multi_class='ovr',
#                                 labels=[1, 2, 3, 4, 5, 6])
# print(round(results['roc_auc'], 2))

In [21]:
# predictions = torch.tensor([
#     # [[1,0,0,0,1], [2,3,2,1,2], [2,2,5,1,3]],
#     [[1,2,0,0,0], [1,float('nan'),1,0,100], [1,4,3,7,10]]
#     ])

# references = torch.tensor([
#     # [0,1,2],
#     [1,4,4]
#     ])

# def softmax(X, axis=0):
#     return np.exp(X)#/np.sum(np.exp(X), axis=axis)

# print(softmax(predictions.numpy(), axis=2))
# print()
# print(predictions.softmax(dim=-1))

# print(predictions.softmax(dim=-1)[0][0])
# compute_metrics((predictions.numpy(), references.numpy()))

In [22]:
data_collator = DataCollatorForTokenClassification(tokenizer=t5_tokenizer)

training_args = TrainingArguments(
    output_dir='./checkpoints',
    learning_rate=src.config.lr,
    per_device_train_batch_size=src.config.batch_size,
    per_device_eval_batch_size=src.config.batch_size,
    num_train_epochs=src.config.num_epochs,
    logging_steps=src.config.logging_steps,
    # save_strategy="steps",
    # save_steps=src.config.save_steps,
    evaluation_strategy="steps",
    eval_steps=src.config.eval_steps,
    # gradient_accumulation_steps=accum,
    # load_best_model_at_end=True,
    # save_total_limit=5,
    seed=42,
    # fp16=True,
    # deepspeed=deepspeed_config,
    remove_unused_columns=False,
    label_names=['labels'],
    # debug="underflow_overflow",
)

trainer = Trainer(
    model=t5_lora_model,
    args=training_args,
    train_dataset=dataset_signalp['train'],
    eval_dataset=dataset_signalp['valid'],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

class EvaluateFirstStepCallback(TrainerCallback):
    def on_step_begin(self, args, state, control, **kwargs):
        if state.global_step == 0:
            control.should_evaluate = True

trainer.add_callback(EvaluateFirstStepCallback())

In [23]:
gc.collect()
torch.cuda.empty_cache()
# torch.mps.empty_cache()

In [24]:
trainer.train()

loss tensor(2.1793e+30, device='cuda:0', grad_fn=<NllLossBackward0>)


Step,Training Loss,Validation Loss,Accuracy Metric,Precision Metric,Recall Metric,F1 Metric,Matthews Correlation,Confusion Matrix
1,2179333899565295953473053917184.000000,7033029923228592081962203611136.000000,0.187847,0.187847,0.187847,0.187847,0.000000,[[ 0 0 0 0 0 0] [ 0 0 0 0 0 0] [ 0 0 0 0 0 0] [202812 6254 7309 54262 16052 3173] [ 0 0 0 0 0 0] [ 0 0 0 0 0 0]]
150,4648222155658315295439389196288.000000,7033029923228592081962203611136.000000,0.187847,0.187847,0.187847,0.187847,0.000000,[[ 0 0 0 0 0 0] [ 0 0 0 0 0 0] [ 0 0 0 0 0 0] [202812 6254 7309 54262 16052 3173] [ 0 0 0 0 0 0] [ 0 0 0 0 0 0]]


KeyboardInterrupt: 

---

In [25]:
metrics=trainer.evaluate()
print(metrics)

KeyboardInterrupt: 

---

In [26]:
training_log = pd.DataFrame(trainer.state.log_history)
display(training_log)

,loss,learning_rate,epoch,step,eval_loss,eval_accuracy_metric,eval_precision_metric,eval_recall_metric,eval_f1_metric,eval_matthews_correlation,eval_confusion_matrix,eval_runtime,eval_samples_per_second,eval_steps_per_second
0,2.179334e+30,0.000100,0.00,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,0.00,1,7.033030e+30,0.187847,0.187847,0.187847,0.187847,0.0,"[[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0...",264.1853,15.705,0.984
2,9.147888e+30,0.000100,0.00,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2.397958e+30,0.000100,0.00,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8.470368e+30,0.000099,0.01,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,4.648222e+30,0.000081,0.19,150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
151,NaN,NaN,0.19,150,7.033030e+30,0.187847,0.187847,0.187847,0.187847,0.0,"[[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0...",264.4193,15.691,0.983
152,5.439923e+30,0.000081,0.19,151,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
153,3.529909e+30,0.000080,0.20,152,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
training_log['eval_confusion_matrix'] = training_log['eval_confusion_matrix'].apply(lambda x: x.tolist() if type(x)==np.ndarray else None)

In [ ]:
adapter_location = '/models/testing_12'
t5_lora_model.save_pretrained(ROOT + adapter_location)
# training_log.to_csv(ROOT + adapter_location + '/training_log.csv', index=False)
training_log.to_parquet(ROOT + adapter_location + '/training_log.parquet')

---

In [ ]:
# for name, param in t5_lora_model.base_model.named_parameters():
#     if "lora" not in name:
#         continue
#     if param.isnan().any():
#         print(f"New parameter {name:<13} | {param.numel():>5} parameters | not updated")
#     else:
#         print(f"New parameter {name:<13} | {param.numel():>5} parameters | updated")

In [ ]:
# params_before = dict(t5_base_model_copy.named_parameters())
# for name, param in t5_lora_model.base_model.named_parameters():
#     if "lora" in name:
#         continue

#     name_before = name.partition(".")[-1].replace("original_", "").replace("module.", "").replace("modules_to_save.default.", "")
#     param_before = params_before[name_before]
#     if torch.allclose(param, param_before):
#         print(f"Parameter {name_before:<14} | {param.numel():>7} parameters | not updated")
#     else:
#         print(f"Parameter {name_before:<14} | {param.numel():>7} parameters | updated")

---

In [ ]:
def predict_model(sequence: str, tokenizer: T5Tokenizer, model: T5EncoderModelForTokenClassification):
    # print('sequence', sequence)
    tokenized_string = tokenizer.encode(sequence, padding=True, truncation=True, return_tensors="pt", max_length=1024)
    # print('tokenized_string', tokenized_string)
    with torch.no_grad():
        output = model(tokenized_string.to(device))
    # print('output', output)
    return output

def translate_logits(logits):
    return [src.config.label_decoding[x] for x in logits.argmax(-1).tolist()[0]]

In [ ]:
_ds_index = 2
_ds_type = 'test'

_inids_test = t5_tokenizer.decode(dataset_signalp[_ds_type][_ds_index]['input_ids'])
_labels_test = dataset_signalp[_ds_type][_ds_index]['labels']
_labels_test_decoded = [src.config.label_decoding[x.item()] for x in _labels_test]
print(_inids_test)
print(_labels_test)
print(_labels_test_decoded)

In [ ]:
preds = predict_model(_inids_test, t5_tokenizer, t5_lora_model)

In [ ]:
_res = translate_logits(preds.logits.cpu().numpy())
print(_res)

In [ ]:
# torch.set_printoptions(threshold=10_000)
# t5_lora_model.custom_classifier.modules_to_save.default.weight

---
---

In [ ]:
t5_base_model_copy.load_adapter(ROOT+adapter_location)

In [ ]:
t5_base_model_copy.to(device)

In [ ]:
preds = predict_model(_inids_test, t5_tokenizer, t5_base_model_copy)

In [ ]:
_res = translate_logits(preds.logits.cpu().numpy())
print(_res)

In [ ]:
print(t5_base_model_copy.custom_classifier.weight)
print(t5_base_model_copy.custom_classifier.bias)

In [ ]:
print(t5_lora_model.model.custom_classifier.modules_to_save.default.weight)
print(t5_lora_model.model.custom_classifier.modules_to_save.default.bias)

In [ ]:
print(t5_lora_model.model.custom_classifier.original_module.weight)
print(t5_lora_model.model.custom_classifier.original_module.bias)

In [ ]:
t5_lora_model.model.custom_classifier

---

In [ ]:
params_trained = [(n, m) for n, m in t5_lora_model.named_parameters() if 'original' not in n]
params_reloaded = [(n, m) for n, m in t5_base_model_copy.named_parameters() if 'original' not in n]

for param_trained, param_reloaded in zip(params_trained, params_reloaded):
    if torch.eq(param_trained[1].data, param_reloaded[1].data).all():
        # print(f"Parameter {param_trained[0]} and {param_reloaded[0]} equal")
        pass
    else:
        print(f"Parameter {param_trained[0]} and {param_reloaded[0]} not equal")

In [ ]:
df = pd.DataFrame({ 'name': [
'Lower Franconia', 'Upper Franconia',
'Middle Franconia', 'Upper Palatinate', 'Swabia', 'Upper Bavaria', 'Lower Bavaria'
], 'capital': [
'Würzburg', 'Bayreuth', 'Ansbach',
'Regensburg', 'Augsburg', 'Munich', 'Landshut' ],
'population': [
1_320_513, 1_061_929, 1_775_169, 1_112_102, 1_899_442, 4_710_865, 1_244_169
], 'area': [
8_530.99, 7_230.19, 7_245.70, 9_692.23, 9_993.97, 17_529.41, 10_329.87 ]
})
df.index = pd.Index(('LF', 'UF', 'MF', 'UP', 'S', 'UB', 'LB'))

---